# QBio202: Deterministic Processes in Biology
# SS 2023
# Exercises 06 (due to 2023-05-24)

***
Ingo Giebel

Student number: 3050992
***

## Task 1.

### Substitutions for nondimensionalization

According to (Fairén, 1979), "all quantities can be rendered dimensionless by convenient change of scales". To this end, in this paper the following substitutions (5a) - (5f) are given:
$$
\begin{align}
    \tau &= k_2t\\
    x &= \frac{k_2}{k_3}X\\
    y &= \frac{k_2}{k_3}Y\\
    a &= \frac{k_5}{k_3}A\\
    b &= \frac{k_1}{k_3}B\\
    q &= \frac{k_3^2k_4}{k_2^2}\\
\end{align}
$$
The above substitutions are rewritten to fit for the equations (2) and (3), as (Fairén, 1979) uses different constant and variable symbols than the symbols used here: $[X]$ instead of $X$, $X$ instead of $x$, $[Y]$ instead of $Y$, $Y$ instead of $y$, $[A]$ instead of $A$, $A$ instead of $a$, $[B]$ instead of $B$, $B$ instead of $b$, $k_2$ instead of $k_1$, $k_3$ instead of $k_2$, $k_4$ instead of $k_3$, $k_5$ instead of $k_4$, $k_1$ instead of $k_5$.

Note that according to (Fairén, 1979) the constants $k_1$, $k_2$, $k_3$, $k_5$ are reaction rates, but $k_4$ is the "dimensional power of the inhibitory step" and therefore **not a reaction rate** as opposed to what is said in the exercise sheet.

### Analysis of the quantity dimensions

Given that $t$ in equation (2) has a time dimension (common unit: $\mathrm{min}$), and $X$, $Y$, $A$, $B$ are concentration variables and constants having a molarity dimension (common unit: $1 \mathrm{M} = 1 \mathrm{mol/L}$), it follows that with the substitutions given in (5):
* $k_1$, $k_2$, $k_5$ have the dimension $\mathrm{time}^{-1}$
* $k_3$ has the dimension $\mathrm{time}^{-1}\mathrm{molarity}^{-1}$
* $k_4$ has the dimension $\mathrm{molarity}^{-1}$
* $\tau = k_2t$ is dimensionless
* $x = \frac{k_2}{k_3}X$ has the dimension $\mathrm{molarity}^2$
* $y = \frac{k_2}{k_3}Y$ has the dimension $\mathrm{molarity}^2$
* $a = \frac{k_5}{k_3}A$ has the dimension $\mathrm{molarity}^2$
* $b = \frac{k_1}{k_3}B$ has the dimension $\mathrm{molarity}^2$
* $q = k_4\frac{k_3^2}{k_2^2}$ has the dimension $\mathrm{molarity}^{-3}$

According to these considerations, the derived quantities are not dimensionless (except $\tau$) as opposed to what is stated in the paper. Yet other papers make the assumption that the concentration variables and constants are given as dimensionless quantities. Under this assumption, the molarity dimension would cancel out, and all the derived variables and constants would indeed be dimensionless.

### Biological interpretation of $a$, $b$ and $q$

* $a$ is the molar increase of metabolite $A$ per time unit divided by the reaction rate constant for the metabolism of the nutrient and oxygen (reaction $X + Y \rightarrow P$)
* $b$ is the molar increase of metabolite $B$ per time unit divided by the reaction rate constant for the metabolism of the nutrient and oxygen
* $q$ is the **inhibitory effect of oxygen** multiplied with the squared quotient of the reaction rate for the metabolism of the nutrient and oxygen divided by the reverse reaction rate of oxygen to metabolite $B$

### Derivation of the nondimensionalized equations

Using the substitutions (5), the equations (2) can be rewritten as follows:
$$
\begin{align}
    \frac{\mathrm{d}X}{\mathrm{d}t} &= k_1B - k_2X - \frac{k_3XY}{(1 + k_4X)^2}\\
    \Longrightarrow \frac{\mathrm{d}(\frac{k_3}{k_2}x)}{\mathrm{d}(\frac{\tau}{k_2})} &= k_1(\frac{k_3}{k_1}b) - k_2(\frac{k_3}{k_2}x) - \frac{k_3(\frac{k_3}{k_2}x)(\frac{k_3}{k_2}y)}{(1 + k_4(\frac{k_3}{k_2}x))^2}\\
    \Longrightarrow k_3\frac{\mathrm{d}x}{\mathrm{d}\tau} &= k_3b - k_3x - k_3\frac{(\frac{k_3}{k_2})^2xy}{(1 + k_4\frac{k_3}{k_2}x)^2}\\
    \Longrightarrow \frac{\mathrm{d}x}{\mathrm{d}\tau} &= b - x - \frac{xy}{(\frac{k_2}{k_3})^2(1 + k_4\frac{k_3}{k_2}x)^2}\\
    \Longrightarrow \frac{\mathrm{d}x}{\mathrm{d}\tau} &= b - x - \frac{xy}{(\frac{k_2}{k_3} + k_4x)^2}\\
\end{align}
$$
The right-hand side of this equation evaluates to $-\frac{1}{4}$ when all variables and constants are set to $1$. On the other, the corresponding equation (2) evaluates to $-\frac{1}{2}$ when all variables and constants are $1$. Therefore, I can not transform the original equations (2) to the nondimensionalized equations (3), using the substitutions given in (Fairén, 1979).